# Numerically determining the attractor stability by noisy update

In [2]:
import sys
sys.path.append('BooleanDOI-master/')
import boolean2_modified as boolean2
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

import BooleanDOI_processing as BDOIp
import BooleanDOI_TargetControl as BDOItc
import BooleanDOI_DOI as BDOI

import cool_bool_tools as cbt

import pystablemotifs as sm
import pyboolnet
import pystablemotifs.export as ex
from collections import Counter


In [3]:
import random

def general_async_pick( lines ):
    line = [ random.choice( lines )]
    #print data
    return line

In [4]:
import noise_config
print(noise_config.epsilon)
def sigmoid(epsilon, x):
    return (1 / (1 + np.exp(-epsilon*(x))) )
print(sigmoid(epsilon=noise_config.epsilon, x=-1))

6.9
0.001006770820085637


In [5]:
p=0.001
x=-1
print(np.log(1-p)-np.log(p))

6.906754778648553


In [6]:
#reading in the rules
model_name='EMT30'
rules=cbt.read_rules_text(model_name)

In [9]:
#format the rules to be PyBoolNet compatible 
rules_pbn = sm.format.booleannet2bnet(rules)
primes = pyboolnet.prime_implicants.bnet_text2primes(rules_pbn)
max_simulate_size=20
ar = sm.AttractorRepertoire.from_primes(primes, max_simulate_size=max_simulate_size)

In [10]:
attractors=(-1+(2*ex.attractor_dataframe(ar).astype(int))).to_dict('index')


## Sampling the noisy state space by running a large number of steps with every attractor being the initial state once. 

In [12]:
large_counter = Counter()
model = boolean2.Model(rules, mode='async')

nr_of_steps=10000 #increase this number to get better convergence

fp_state_dict={}
for attr_index,start_state in attractors.items():
    print(attr_index)
    model.initialize(lambda node: start_state[node])
    model.iterate(nr_of_steps,shuffler=general_async_pick)
    #print('Percentage of all states visited: ',float(len(model.states))/2**len(model.nodes))
    fp_states=model.fp()
    state_occurances=Counter(fp_states)
    print('Percentage of all states visited: ',float(len(state_occurances))/2**len(model.nodes))
    #edge_occurances=Counter([(fp_states[i-1], fp_states[i]) for i in range(len(fp_states))])
    for i,fps in enumerate(model.fp()):
        fp_state_dict[fps]=model.states[i]
    large_counter+=state_occurances

0
Percentage of all states visited:  2.60770320892334e-08
1
Percentage of all states visited:  3.91155481338501e-08
2
Percentage of all states visited:  3.4458935260772705e-08
3
Percentage of all states visited:  6.05359673500061e-08
4
Percentage of all states visited:  5.3085386753082275e-08
5
Percentage of all states visited:  2.0489096641540527e-08
6
Percentage of all states visited:  1.1175870895385742e-08


In [13]:
assert sum(large_counter.values())==len(attractors)*(nr_of_steps+1)


In [14]:
total_nr_of_steps=sum(large_counter.values())

In [15]:
attr_probabilities={}
for i,j in large_counter.most_common():
    attr_alert='not attractor'
    for a_index,state in attractors.items():
        
        if state==dict(fp_state_dict[i]):
            attr_alert='Attractor ',a_index
            attr_probabilities[a_index]=float(j)/total_nr_of_steps
            print(i, float(j)/total_nr_of_steps, attr_alert)
    

0 0.47655234476552344 ('Attractor ', 0)
189 0.1326295941834388 ('Attractor ', 5)
28 0.10684645821132173 ('Attractor ', 1)
207 0.0986329938434728 ('Attractor ', 6)
98 0.07222134929364206 ('Attractor ', 3)
66 0.033568071764252146 ('Attractor ', 2)
152 0.004770951476280943 ('Attractor ', 4)


In [16]:
df_attr=ex.attractor_dataframe(ar)

In [17]:
df_attr['attr_stability']=df_attr.index.map(attr_probabilities)

In [18]:
df_attr

,AKT,AXIN2_cyto,AXIN2_nuc,Bcatenin_memb,Bcatenin_nuc,Dest_compl,Ecadherin,Ecadherin_CTF,GLI_cyto,GLI_nuc,...,SNAI2_nuc,SOS_GRB2,TGFBR,TGFBR_icd,TWIST1_nuc,ZEB1_cyto,ZEB1_nuc,ZEB2,miR200,attr_stability
0,1,1,0,0,1,0,0,1,1,1,...,1,0,1,1,1,1,1,1,0,0.476552
1,0,1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0.106846
2,0,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0.033568
3,0,1,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0.072221
4,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0.004771
5,0,0,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0.132630
6,0,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0.098633


In [19]:
df_attr.to_excel('model_%s_attr_probabilities_p_error_%s.xlsx'%(model_name,str(p)))